<h1>This tool allows you to duplicate a layer with a new name!</h1>

In [ ]:
#Import necessary layers
#!pip install LMIPy
import LMIPy as lmi
import os
import json
import geopandas as gpd
from IPython.display import display
from pprint import pprint
from datetime import datetime
import numpy as np
import json

<font color=red>**ACTION REQUIRED** <br>
<font color=blue>**Enter your API Token Below**</font>

In [ ]:
#API token needed to make changes
API_TOKEN = ''


<font color=red>**ACTION REQUIRED** <br>
<font color=blue>**Enter your dataset ID below** <br>
The dataset will be printed out below, then the first layer in the list of layers, followed by the number of layers in the dataset. We will duplicate the first layer.</font>

In [ ]:
#Select Dataset and Layer to Clone, here we just chose the first layer
dataset_id = '9e9a5c50-b825-4f12-838f-1650943c2be1' #NEEDS TO BE THE LONG ONE LIKE 6cfd6255-609e-4922-8709-5aaad0db9fae
dataset = lmi.Dataset(dataset_id) #select the dataset by changing the ID
long_dataset_id = dataset.id

print('Dataset selected is below: ')
display(dataset)

layer_to_clone = dataset.layers[0]   #Chose the first layer

print('First layer is below:')
display(layer_to_clone)

print("Number of layers in dataset: {}".format(len(dataset.layers)))


__________________________________________________________________________________________________________________
<font color=blue>**Now we will get the attributes that need to change**<br>
This includes the layer name, description, SQL statement, timeline order, timeline label, and interaction <br>
We'll first print them out so you can see</font>

In [ ]:
#Gathering the right attributes that need to change
name = layer_to_clone.attributes['name']
description = layer_to_clone.attributes['description']
appConfig = layer_to_clone.attributes['layerConfig']



try:
    sql = appConfig['body']['layers'][0]['options']['sql']
except: 
    sql = None
    print('SQL not found')
try:
    order = str(appConfig['order'])
except:
    order = None
    print('Order not found')
try:
    timeLineLabel = None
    timeLineLabel = appConfig['timelineLabel']
except:
    print('Timeline not found')
try:
    interactionConfig = None
    interactionConfig = layer_to_clone.attributes['interactionConfig']
except:
    print('Interaction not found')

#Print them out
print('Name: \n{} \n'.format(name))
print('Description: \n{} \n'.format(description))
print('SQL Statement: \n{} \n'.format(sql))
print('Order Value: \n{} \n'.format(order))
print('Timeline Label: \n{} \n'.format(timeLineLabel))
print('Interaction Config: \n{} \n'.format(interactionConfig))

<h1>Here we will replace the values in the attributes and decide what to loop over.</h1>

<font color=red>**ACTION REQUIRED**</font> <br>
<font color=blue>**Enter the text you would like to replace in the attributes above in the variable 'replace_string'** <br>
**AND enter the list of values you'd like to replace the search string with in the variable 'loop_values'.**<br>
The next cell will search for that string and replace it with a {} so it can be filled in later.<br>
Then we will loop over the values in 'loop_values' and replace the {} in the attributes with the values.</font>

In [ ]:
#Replace this value with the string you'd like to replace in the attributes

new_name = name+'_copy'
new_description = description
new_sql = sql
new_order = order
new_timeLineLabel = timeLineLabel
new_interactionConfig = interactionConfig

<font color=blue>**Above are the values you entered. Double check that they look okay.**</font>

<font color=blue>**Next we will print out the attributes after replacing the 'replace_string' with '{}'**</font>

In [ ]:
#Print them out
print('New Name: \n{} \n'.format(new_name))
print('New Description: \n{} \n'.format(new_description))
print('New SQL Statement: \n{} \n'.format(new_sql))
print('New Order Value: \n{} \n'.format(new_order))
print('New Timeline Label: \n{} \n'.format(new_timeLineLabel))
print('New Interaction Config: \n{} \n'.format(new_interactionConfig))

<font color=blue>**Check out the values above, do they look okay? If not you can manually replace the values above.**</font>

In [ ]:
#If any of the variables above are not okay, replace it in the cell above

<h1>Now we will create the new layer!</h1>

<font color=blue>**When running the cell bellow, it sometimes returns an error that it times out, just run the cell again.**</font>

In [ ]:
#SOMETIMES WE GET AN ERROR THAT IT TIMES OUT, JUST RUN THE CELL AGAIN

#Load in dataset again to have the most updated version
dataset = lmi.Dataset(dataset_id)

#Create dictionary with new layer name and description
clone_attributes = {
            'name': new_name,
            'description': new_description,
            'published':False
}
#Clone old layer with new name and description
new_layer = layer_to_clone.clone(token=API_TOKEN, env='preproduction', layer_params=clone_attributes,
                                 target_dataset_id=dataset_id)

#Get current layer configuration
appConfig = new_layer.attributes['layerConfig']
#And replace with new values
if new_sql:
    appConfig['body']['layers'][0]['options']['sql'] = new_sql
if new_order:
    appConfig['order'] = new_order
if new_timeLineLabel:
    appConfig['timelineLabel'] = new_timeLineLabel  
payload = {
    'layerConfig': {
        **appConfig
    }
}
#Push updates to new layer
new_layer = new_layer.update(update_params=payload, token=API_TOKEN)
     
    
#Replace interaction config with new values
payload = {
    'interactionConfig': {
        **new_interactionConfig
        }
}
#Push update to layer
new_layer = new_layer.update(update_params=payload, token=API_TOKEN)

#Double check that we renamed the layer
payload = {
    'name': new_name,
    'description': new_description
    }
new_layer = new_layer.update(update_params=payload, token=API_TOKEN)
print('Success! Created new layer:')
print(new_layer)
        

In [ ]:
#Use this section to delete layers if they are stuck in staging or you want to bulk delete layers

# #UNCOMMENT THIS IF YOU WANT TO DELETE A SPECIFIC LAYER
# #Select layer by layer id:
# layer = lmi.Layer('c3075c5a-5567-4b09-bc0d-96ed1673f8b6')
# layer.delete(token=API_TOKEN)

# #UNCOMMENT THIS SECTION IF YOU WANT TO DELETE ALL LAYERS EXCEPT THE FIRST ONE
# #Enter dataset id (the long one)
# dataset_id = 'e32bdaba-5a72-487c-b640-668adaea2976' #THIS NEEDS TO BE THE LONG STRING ONE
# dataset = lmi.Dataset(dataset_id) 
# for i, layer in enumerate(dataset.layers):
#     if i<len(dataset.layers)-1:
#         display(layer)
#         layer.delete(token=API_TOKEN)